In [28]:
from imutils.perspective import four_point_transform
from skimage.segmentation import clear_border
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import imutils
import cv2 as cv
from sudoku import Sudoku

In [29]:
img = cv.imread("1.jpg", cv.IMREAD_COLOR)
img = imutils.resize(img, width = 600)

In [30]:
cv.imshow("", img)
cv.waitKey(0)

-1

In [31]:
def boundaries(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    blurred = cv.GaussianBlur(gray, (7,7), 3)
    threshold = cv.adaptiveThreshold(blurred, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11,2)
    threshold = cv.bitwise_not(threshold)
    cv.imshow("", threshold)
    cv.waitKey(0)
    contour = cv.findContours(threshold.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    contour = imutils.grab_contours(contour)
    contour = sorted(contour, key = cv.contourArea, reverse=True)
    count = None
    
    for c in contour:
        length = cv.arcLength(c, True)
        
        approx = cv.approxPolyDP(c, 0.02 * length, True)
        
        if len(approx) == 4:
            count = approx
            print(length)
            print(approx)
            break
        
    if count is None:
        print("No puzzle found")
        
    img = image.copy()
    cv.drawContours(img, [count], -1, (0,255,0),2)
    cv.imshow("",img)
    cv.waitKey(0)
    print(count)
    puzzle = four_point_transform(image, count.reshape(4,2))
    warped = four_point_transform(gray, count.reshape(4,2)) 
    cv.imshow("",puzzle)
    cv.waitKey(0)
    return (puzzle, warped)

In [39]:
color, grayscale = boundaries(img)

2074.416301369667
[[[ 38  91]]

 [[ 88 628]]

 [[552 565]]

 [[568  92]]]
[[[ 38  91]]

 [[ 88 628]]

 [[552 565]]

 [[568  92]]]


In [32]:
def extract(cell):
    threshold = cv.threshold(cell, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)[1]
    print(threshold.shape)
    #cv.imshow("",threshold)
    #cv.waitKey(0)
    threshold = clear_border(threshold)
    #cv.imshow("",threshold)
    #cv.waitKey(0)
    contour = cv.findContours(threshold.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    contour = imutils.grab_contours(contour)
    img = cell.copy()
    cv.drawContours(img, contour, -1, (0,255,0),2)
    #cv.imshow("",img)
    #cv.waitKey(0)
    if len(contour) == 0:
        return None
    
    c = max(contour, key = cv.contourArea)
    mask = np.zeros(threshold.shape).astype("int8")
    cv.drawContours(mask, [c], -1, 255, -1)
    #cv.imshow("",mask)
    #cv.waitKey(0)
    (height, width) = threshold.shape
    filled = cv.countNonZero(mask)/float(height*width)
    
    if filled < 0.03:
        return None
    
    digit = cv.bitwise_and(threshold, threshold, mask=mask)
    #cv.imshow("", digit)
    #cv.waitKey(0)
    return digit

In [10]:
extract(grayscale)

(572, 572)


In [11]:
grey.shape[0]

NameError: name 'grey' is not defined

In [33]:

digit_classifier = load_model("digit_classifier")
(colored, grey) = boundaries(img)
board = np.zeros((9,9), dtype = "int")
X_size = grey.shape[1] // 9
y_size = grey.shape[0] // 9

cells = []

for y in range(0,9):
    row = []
    for x in range(0,9):
        startX = x * X_size
        startY = y * y_size
        endX = (x+1) * X_size
        endY = (y+1) * y_size
        row.append((startX, startY, endX, endY))
        print((startX, startY, endX, endY))
        cell = grey[startY:endY, startX:endX]
        digit = extract(cell)
        
        if digit is not None:
            area = cv.resize(digit, (28,28))
            area = area.astype("float") / 255.0
            area = img_to_array(area)
            area = np.expand_dims(area, axis = 0)
            prediction = digit_classifier.predict(area).argmax(axis=1)[0]
            print(prediction)
            board[y,x] = prediction
    cells.append(row)
        

2074.416301369667
[[[ 38  91]]

 [[ 88 628]]

 [[552 565]]

 [[568  92]]]
[[[ 38  91]]

 [[ 88 628]]

 [[552 565]]

 [[568  92]]]
(0, 0, 58, 59)
(59, 58)
(58, 0, 116, 59)
(59, 58)
7
(116, 0, 174, 59)
(59, 58)
(174, 0, 232, 59)
(59, 58)
(232, 0, 290, 59)
(59, 58)
2
(290, 0, 348, 59)
(59, 58)
(348, 0, 406, 59)
(59, 58)
(406, 0, 464, 59)
(59, 58)
4
(464, 0, 522, 59)
(59, 58)
6
(0, 59, 58, 118)
(59, 58)
(58, 59, 116, 118)
(59, 58)
6
(116, 59, 174, 118)
(59, 58)
(174, 59, 232, 118)
(59, 58)
(232, 59, 290, 118)
(59, 58)
(290, 59, 348, 118)
(59, 58)
(348, 59, 406, 118)
(59, 58)
8
(406, 59, 464, 118)
(59, 58)
9
(464, 59, 522, 118)
(59, 58)
(0, 118, 58, 177)
(59, 58)
2
(58, 118, 116, 177)
(59, 58)
(116, 118, 174, 177)
(59, 58)
(174, 118, 232, 177)
(59, 58)
8
(232, 118, 290, 177)
(59, 58)
(290, 118, 348, 177)
(59, 58)
(348, 118, 406, 177)
(59, 58)
7
(406, 118, 464, 177)
(59, 58)
1
(464, 118, 522, 177)
(59, 58)
5
(0, 177, 58, 236)
(59, 58)
(58, 177, 116, 236)
(59, 58)
8
(116, 177, 174, 236)
(59, 

In [34]:
print(board)

[[0 7 0 0 2 0 0 4 6]
 [0 6 0 0 0 0 8 9 0]
 [2 0 0 8 0 0 7 1 5]
 [0 8 4 0 9 7 0 0 0]
 [7 1 0 0 0 0 0 5 9]
 [0 0 0 1 3 0 4 8 0]
 [6 9 7 0 0 2 0 0 8]
 [0 5 8 0 0 0 0 6 0]
 [4 3 0 0 8 0 0 7 0]]


In [35]:
solution = Sudoku(3,3, board = board.tolist())
solution = solution.solve()
print(solution.board)

[[8, 7, 5, 9, 2, 1, 3, 4, 6], [3, 6, 1, 7, 5, 4, 8, 9, 2], [2, 4, 9, 8, 6, 3, 7, 1, 5], [5, 8, 4, 6, 9, 7, 1, 2, 3], [7, 1, 3, 2, 4, 8, 6, 5, 9], [9, 2, 6, 1, 3, 5, 4, 8, 7], [6, 9, 7, 4, 1, 2, 5, 3, 8], [1, 5, 8, 3, 7, 9, 2, 6, 4], [4, 3, 2, 5, 8, 6, 9, 7, 1]]


In [37]:
test = color
color = test

In [40]:

for (cell, sol) in zip(cells, solution.board):
    for(box, digit) in zip(cell, sol):
        X1, Y1, X2, Y2 = box
        X = int((X2-X1) * 0.33)
        Y = int((Y2-Y1) * -0.2)
        X += X1
        Y += Y2
        print(box)
        cv.putText(color, str(digit), (X,Y), cv.FONT_HERSHEY_SIMPLEX,0.9, (0,255,0),2)
cv.imshow("", color)
cv.waitKey(0)

(0, 0, 58, 59)
(58, 0, 116, 59)
(116, 0, 174, 59)
(174, 0, 232, 59)
(232, 0, 290, 59)
(290, 0, 348, 59)
(348, 0, 406, 59)
(406, 0, 464, 59)
(464, 0, 522, 59)
(0, 59, 58, 118)
(58, 59, 116, 118)
(116, 59, 174, 118)
(174, 59, 232, 118)
(232, 59, 290, 118)
(290, 59, 348, 118)
(348, 59, 406, 118)
(406, 59, 464, 118)
(464, 59, 522, 118)
(0, 118, 58, 177)
(58, 118, 116, 177)
(116, 118, 174, 177)
(174, 118, 232, 177)
(232, 118, 290, 177)
(290, 118, 348, 177)
(348, 118, 406, 177)
(406, 118, 464, 177)
(464, 118, 522, 177)
(0, 177, 58, 236)
(58, 177, 116, 236)
(116, 177, 174, 236)
(174, 177, 232, 236)
(232, 177, 290, 236)
(290, 177, 348, 236)
(348, 177, 406, 236)
(406, 177, 464, 236)
(464, 177, 522, 236)
(0, 236, 58, 295)
(58, 236, 116, 295)
(116, 236, 174, 295)
(174, 236, 232, 295)
(232, 236, 290, 295)
(290, 236, 348, 295)
(348, 236, 406, 295)
(406, 236, 464, 295)
(464, 236, 522, 295)
(0, 295, 58, 354)
(58, 295, 116, 354)
(116, 295, 174, 354)
(174, 295, 232, 354)
(232, 295, 290, 354)
(290, 295,

-1

In [ ]:
len(cells[0])

In [ ]:
np.asarray(cells).shape

In [ ]:
for (cell, sol) in zip(cells, solution.board):
    for(box, digit) in zip(cell, sol):
        print(box, digit)